# ファインチューニング - 入門 Hugging Face🤗

[GitHub](https://github.com/tpu-dsg/hf-hands-on)

このノートブックでは、[Hugging Face🤗](https://huggingface.co/)のエコシステムを活用して、ファインチューニングによる画像分類を行います。

NOTICE: [Hugging Faceのガイド](https://huggingface.co/docs/transformers/ja/tasks/image_classification)を参考に作成されました。

--

Colabのみ:以下をコメントアウトのうえ実行して、依存するパッケージを更新してください。また、必要に応じてドライブをマウントしてください。

In [ ]:
# !pip install -U -q transformers
# !pip install -q evaluate==0.4.3
# !pip uninstall -y wandb

# 任意でドライブをマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import evaluate
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
)
from datasets import load_dataset
from huggingface_hub import notebook_login
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
# 使用するモデル
CHECKPOINT: str = "google/vit-base-patch16-224-in21k"
# 訓練後のモデルの名前
TUNED_MODEL_NAME: str = "my_awesome_food_model"

In [ ]:
image_processor = AutoImageProcessor.from_pretrained(CHECKPOINT)

## データセットの準備

Food-101データセットのサブセットをロードし、データセットの`train`をtrainセットとtestセットに分割します。

In [ ]:
food = load_dataset("food101", split="train[:5000]")
food = food.train_test_split(test_size=0.2)

一つデータを見てみましょう。

In [ ]:
food["train"][0]

データセット内の各例には 2 つのフィールドがあります。

- `image`: 食品の PIL 画像
- `label`: 食品のラベルクラス

モデルがラベル ID からラベル名を取得しやすくするために、ラベル名をマップする辞書を作成します。 

In [ ]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

これで、ラベル ID をラベル名に変換できるようになりました。

In [ ]:
id2label[str(79)]

[albumentations](https://albumentations.ai/)を使用したデータ拡張を定義します。

In [ ]:
_transforms = A.Compose(
    [
        A.Resize(image_processor.size["height"], image_processor.size["width"]),
        A.RandomCrop(image_processor.size["height"], image_processor.size["width"]),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=image_processor.image_mean, std=image_processor.image_std),
        ToTensorV2(),
    ]
)

次に、変換を適用し画像の`pixel_values`(モデルへの入力) を返す前処理関数を作成します。

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [
        _transforms(image=np.array(img.convert("RGB")))["image"]
        for img in examples["image"]
    ]
    del examples["image"]
    return examples

In [ ]:
food = food.with_transform(transforms)
data_collator = DefaultDataCollator()

## 学習の設定

メトリクスの計算方法を定義

今回は正解率を最大化するようにします。

In [ ]:
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

モデルをロード

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    CHECKPOINT,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

トレーニング引数をTrainerに渡します。

Tips: `push_to_hub=True`を設定すると、このモデルをHubにプッシュできます。(Huffing Faceへのログインが必要です):

In [ ]:
# notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir=TUNED_MODEL_NAME,
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    processing_class=image_processor,
    compute_metrics=compute_metrics,
)

## 学習

`TUNED_MODEL_NAME`に指定した名前のディレクトリにチェックポイントが保存されます。

In [ ]:
trainer.train()

モデルの保存

In [ ]:
trainer.save_model("./" + TUNED_MODEL_NAME)

`push_to_hub=True`を設定し、ログイン済みであれば、以下を実行することでモデルを公開できます。

In [ ]:
# trainer.push_to_hub()

## 推論

データの読み込み

In [ ]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]
image

先ほど学習したモデルを使用して推論を実行します。

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("./" + TUNED_MODEL_NAME)
inputs = image_processor(image, return_tensors="pt")

model = AutoModelForImageClassification.from_pretrained("./" + TUNED_MODEL_NAME)

with torch.no_grad():
    logits = model(**inputs).logits

結果の表示

In [ ]:
predicted_label = logits.argmax(-1).item()

model.config.id2label[predicted_label]